In [2]:
import numpy as np
from scipy import signal
import torch
from skimage.measure import block_reduce

In [15]:
model = torch.load("lenet5_mnist_model.pth")

model_state = torch.load("lenet5_mnist_model.pth", map_location=torch.device('cpu'))
conv1_weight = model_state['conv1.weight'].numpy()
conv1_bias = model_state['conv1.bias'].numpy()
conv2_weight = model_state['conv2.weight'].numpy()
conv2_bias = model_state['conv2.bias'].numpy()
fc1_weight = model_state['fc1.weight'].numpy()
fc1_bias = model_state['fc1.bias'].numpy()
fc2_weight = model_state['fc2.weight'].numpy()
fc2_bias = model_state['fc2.bias'].numpy()
fc3_weight = model_state['fc3.weight'].numpy()
fc3_bias = model_state['fc3.bias'].numpy()


# --- Define constants to match C headers ---
FILTER_SIZE = 5
NUM_FILTER_ONE = 6
NUM_FILTER_TWO = 16
INPUT_SIZE_ONE = 28
OUTPUT_SIZE_ONE = 24
POOL_OUTPUT_SIZE_ONE = 12
INPUT_SIZE_TWO = 12
OUTPUT_SIZE_TWO = 8
POOL_OUTPUT_SIZE_TWO = 4
FC_LAYER_SIZE_ONE = 256 # 16 * 4 * 4
FC_LAYER_SIZE_TWO = 120
FC_LAYER_SIZE_THREE = 84
FC_LAYER_SIZE_FOUR = 10
POOL_SIZE = 2


/var/folders/hv/4b688pdd2s33m7ndy82kcf6c0000gn/T/ipykernel_51216/3495782225.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("lenet5_mnist_model.pth")


In [16]:
def print_feature_map_corner(feature_map, name, map_idx):
    print(f"  {name} {map_idx} (Top-Left Corner):")
    np.set_printoptions(precision=6, suppress=True, floatmode='fixed', sign=' ')
    print_rows = min(feature_map.shape[0], 5)
    print_cols = min(feature_map.shape[1], 5)
    corner = feature_map[:print_rows, :print_cols]
    for row in corner:
        print("    ", " ".join([f"{x:+10.6f}" for x in row]))
    print()

def print_output_vector(vector, name):
    print(f"  {name}:")
    np.set_printoptions(precision=6, suppress=True, floatmode='fixed')
    print(f"    [{', '.join([f'{x:8.6f}' for x in vector])}]")
    print()

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

In [17]:
# --- 1. Prepare Test Input ---
print("--- Preparing Test Input ---")
input_one = np.zeros((INPUT_SIZE_ONE, INPUT_SIZE_ONE), dtype=np.float32)
for i in range(INPUT_SIZE_ONE):
    for j in range(INPUT_SIZE_ONE):
        input_one[i, j] = (i + j) / 100.0
print("Input prepared.\n")

--- Preparing Test Input ---
Input prepared.



In [18]:
# --- 2. CONV1 -> RELU -> POOL1 Pipeline ---
print("--- Running CONV1 ---")
output_one = np.zeros((NUM_FILTER_ONE, OUTPUT_SIZE_ONE, OUTPUT_SIZE_ONE), dtype=np.float32)
for f_idx in range(NUM_FILTER_ONE):
    kernel = conv1_weight[f_idx, 0, :, :]
    conv_result = signal.correlate2d(input_one, kernel, mode='valid')
    np.add(conv_result, conv1_bias[f_idx], out=output_one[f_idx, :, :])
print("CONV1 complete. Printing results (pre-activation)...\n")
print_feature_map_corner(output_one[0], "Feature Map (Pre-ReLU)", 0)

--- Running CONV1 ---
CONV1 complete. Printing results (pre-activation)...

  Feature Map (Pre-ReLU) 0 (Top-Left Corner):
      -0.097587  -0.091909  -0.086230  -0.080551  -0.074873
      -0.091909  -0.086230  -0.080551  -0.074873  -0.069194
      -0.086230  -0.080551  -0.074873  -0.069194  -0.063515
      -0.080551  -0.074873  -0.069194  -0.063515  -0.057837
      -0.074873  -0.069194  -0.063515  -0.057837  -0.052158



In [19]:
print("--- Applying ReLU to CONV1 Output ---")
output_one_relu = np.maximum(0, output_one)
print("ReLU complete. Printing results...\n")
print_feature_map_corner(output_one_relu[0], "Feature Map (Post-ReLU)", 0)

--- Applying ReLU to CONV1 Output ---
ReLU complete. Printing results...

  Feature Map (Post-ReLU) 0 (Top-Left Corner):
      +0.000000  +0.000000  +0.000000  +0.000000  +0.000000
      +0.000000  +0.000000  +0.000000  +0.000000  +0.000000
      +0.000000  +0.000000  +0.000000  +0.000000  +0.000000
      +0.000000  +0.000000  +0.000000  +0.000000  +0.000000
      +0.000000  +0.000000  +0.000000  +0.000000  +0.000000



In [20]:
print("--- Running POOL1 ---")
pool_one_output = block_reduce(output_one_relu, block_size=(1, POOL_SIZE, POOL_SIZE), func=np.max).squeeze()
print("POOL1 complete. Printing results...\n")
print_feature_map_corner(pool_one_output[0], "Pooled Map", 0)

--- Running POOL1 ---
POOL1 complete. Printing results...

  Pooled Map 0 (Top-Left Corner):
      +0.000000  +0.000000  +0.000000  +0.000000  +0.000000
      +0.000000  +0.000000  +0.000000  +0.000000  +0.000000
      +0.000000  +0.000000  +0.000000  +0.000000  +0.000000
      +0.000000  +0.000000  +0.000000  +0.000000  +0.000000
      +0.000000  +0.000000  +0.000000  +0.000000  +0.004629



In [21]:
# --- 3. CONV2 -> RELU -> POOL2 Pipeline ---
print("--- Running CONV2 ---")
output_two = np.zeros((NUM_FILTER_TWO, OUTPUT_SIZE_TWO, OUTPUT_SIZE_TWO), dtype=np.float32)
for f_idx in range(NUM_FILTER_TWO):
    total_conv_result = np.zeros((OUTPUT_SIZE_TWO, OUTPUT_SIZE_TWO), dtype=np.float32)
    for c_idx in range(NUM_FILTER_ONE):
        input_channel = pool_one_output[c_idx, :, :]
        kernel = conv2_weight[f_idx, c_idx, :, :]
        conv_result_inner = signal.correlate2d(input_channel, kernel, mode='valid')
        np.add(total_conv_result, conv_result_inner, out=total_conv_result)
    np.add(total_conv_result, conv2_bias[f_idx], out=output_two[f_idx, :, :])
print("CONV2 complete. Printing results (pre-activation)...\n")
print_feature_map_corner(output_two[0], "Feature Map (Pre-ReLU)", 0)

--- Running CONV2 ---
CONV2 complete. Printing results (pre-activation)...

  Feature Map (Pre-ReLU) 0 (Top-Left Corner):
      +0.041441  +0.041686  +0.041956  +0.042058  +0.042791
      +0.041686  +0.041956  +0.042058  +0.042791  +0.044814
      +0.041956  +0.042058  +0.042791  +0.044814  +0.045892
      +0.042058  +0.042791  +0.044814  +0.045892  +0.045218
      +0.042791  +0.044814  +0.045892  +0.045218  +0.042223



In [22]:
print("--- Applying ReLU to CONV2 Output ---")
output_two_relu = np.maximum(0, output_two)
print("ReLU complete. Printing results...\n")
print_feature_map_corner(output_two_relu[0], "Feature Map (Post-ReLU)", 0)

--- Applying ReLU to CONV2 Output ---
ReLU complete. Printing results...

  Feature Map (Post-ReLU) 0 (Top-Left Corner):
      +0.041441  +0.041686  +0.041956  +0.042058  +0.042791
      +0.041686  +0.041956  +0.042058  +0.042791  +0.044814
      +0.041956  +0.042058  +0.042791  +0.044814  +0.045892
      +0.042058  +0.042791  +0.044814  +0.045892  +0.045218
      +0.042791  +0.044814  +0.045892  +0.045218  +0.042223



In [23]:
print("--- Running POOL2 ---")
pool_two_output = block_reduce(output_two_relu, block_size=(1, POOL_SIZE, POOL_SIZE), func=np.max).squeeze()
print("POOL2 complete. Printing results...\n")
print_feature_map_corner(pool_two_output[0], "Pooled Map", 0)

--- Running POOL2 ---
POOL2 complete. Printing results...

  Pooled Map 0 (Top-Left Corner):
      +0.041956  +0.042791  +0.045892  +0.045892
      +0.042791  +0.045892  +0.045892  +0.042223
      +0.045892  +0.045892  +0.042223  +0.034246
      +0.045892  +0.042223  +0.034246  +0.026270



In [31]:
# --- 4. Flatten and Dense Layers ---
print("--- Flattening Output for Dense Layers ---")
dense_one_input = pool_two_output.flatten()
print(f"Flattening complete. Input size for FC1: {dense_one_input.shape[0]}\n")
print_output_vector(dense_one_input, "Logits (INITIAL)")

--- Flattening Output for Dense Layers ---
Flattening complete. Input size for FC1: 256

  Logits (INITIAL):
    [0.041956, 0.042791, 0.045892, 0.045892, 0.042791, 0.045892, 0.045892, 0.042223, 0.045892, 0.045892, 0.042223, 0.034246, 0.045892, 0.042223, 0.034246, 0.026270, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.0000

In [29]:
print("--- Running DENSE1 -> RELU ---")
dense_one_output = fc1_weight @ dense_one_input + fc1_bias
dense_one_output_relu = np.maximum(0, dense_one_output)
print_output_vector(dense_one_output_relu, "Logits (FC1)")

--- Running DENSE1 -> RELU ---
  Logits (FC1):
    [0.018149, 0.000000, 0.007517, 0.000000, 0.000000, 0.124135, 0.066744, 0.021024, 0.000000, 0.000000, 0.000000, 0.000000, 0.002011, 0.072472, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.017108, 0.000000, 0.000000, 0.042124, 0.020060, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.076161, 0.000000, 0.000000, 0.037448, 0.010059, 0.027473, 0.029273, 0.036703, 0.073573, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.047025, 0.027201, 0.097104, 0.038254, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.001817, 0.035273, 0.000000, 0.000000, 0.000000, 0.039531, 0.000000, 0.103109, 0.000000, 0.000000, 0.090636, 0.000000, 0.041314, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.005445, 0.089195, 0.076087, 0.000000, 0.024069, 0.000000, 0.023788, 0.047666, 0.004387, 0.011409, 0.018119, 0.000000, 0.000000, 0.014813, 0.014606, 0.008696, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.061535, 0.000000, 0.000000

In [30]:
print("--- Running DENSE2 -> RELU ---")
dense_two_output = fc2_weight @ dense_one_output_relu + fc2_bias
dense_two_output_relu = np.maximum(0, dense_two_output)
print_output_vector(dense_two_output_relu, "Logits (FC2)")


--- Running DENSE2 -> RELU ---
  Logits (FC2):
    [0.051727, 0.119112, 0.236509, 0.026136, 0.000000, 0.142423, 0.006859, 0.104868, 0.168603, 0.002270, 0.000000, 0.069113, 0.000000, 0.025621, 0.048301, 0.208578, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.179778, 0.000000, 0.000000, 0.000000, 0.171219, 0.000000, 0.102268, 0.054697, 0.144813, 0.013344, 0.018867, 0.000000, 0.056256, 0.022479, 0.000000, 0.000000, 0.099911, 0.000000, 0.000000, 0.117540, 0.189064, 0.000000, 0.065537, 0.113191, 0.000000, 0.000000, 0.031571, 0.121755, 0.000000, 0.021920, 0.100286, 0.000000, 0.142740, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.034261, 0.150238, 0.119610, 0.014704, 0.056657, 0.000000, 0.176617, 0.000000, 0.000000, 0.000000, 0.166932, 0.086276, 0.149714, 0.135327, 0.081560, 0.147315, 0.102524, 0.000000, 0.039621, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000]



In [27]:
print("--- Running DENSE3 (Output Layer) ---")
final_output_logits = fc3_weight @ dense_two_output_relu + fc3_bias
print_output_vector(final_output_logits, "Logits (Output Before Softmax)")

--- Running DENSE3 (Output Layer) ---
  Logits (Output Before Softmax):
    [-0.391998, 0.244302, -0.370132, -0.183586, 0.010328, -0.173416, -0.232156, -0.044213, -0.000342, -0.016745]



In [28]:
print("--- Applying Softmax ---")
final_probabilities = softmax(final_output_logits)
print_output_vector(final_probabilities, "Final Probabilities")

--- Applying Softmax ---
  Final Probabilities:
    [0.074588, 0.140933, 0.076237, 0.091872, 0.111532, 0.092811, 0.087516, 0.105611, 0.110348, 0.108553]

